<div>
<img src="https://www.sasview.org/img/sasview_logo.png" width="300"/>
</div>

# Jupyter Notebook Sasmodels Scripting Tutorial
##  Contributor Camp, January 20, 2024
----  
### Caitlyn Wolf
NRC Postdoc, Chemical Engineer  
NIST Center for Neutron Research

## What is the difference between SasView and sasmodels?

[SasView](https://www.sasview.org/), more broadly, is a software pacakge that enables analysis of small angle scattering data by inverse space data modeling of 1D and 2D scattering data. It includes over 70 built-in models, the ability to apply polydispersity and resolution, fitting with your custom models and many more scattering analysis tools.

The software pacakge is actually broken up into multiple github repositories including [sasview](https://github.com/SasView/sasview), [sasmodels](https://github.com/SasView/sasmodels), and [sasdata](https://github.com/SasView/sasdata). The sasview library includes code for the GUI while the sasmodels library includes code for all the models, polydispersity, and resolution calculations. It also interfaces with [bumps](https://github.com/bumps), a data fitting library. Sasdata handles the import and export of small-angle scattering data, both in standard data formats from multiple facilities as well as more general file formats (e.g., txt or csv).

The sasmodels and bumps libraries can be imported and used directly into Python scripts, enabling the development of custom and reproducible analysis procedures for large amounts of scattering data. In this tutorial we will primarily be using these two pacakges, however we will incorporate sasdata later on!







#  1.&nbsp;Python Basics
In this section we will learn about some basic Python concepts while loading some scattering data.

## 1.1 data types



![Python data types](https://media.geeksforgeeks.org/wp-content/uploads/20191023173512/Python-data-structure.jpg)  
Image source: https://www.geeksforgeeks.org/python-data-types/

## 1.2 importing libraries

To access functionality in other Python packages, we need to use the `import` statement. There are a few packages that were automatically installed in our Python environment in Google Colab.

In [ ]:
!pip list

Let's import the `numpy` package. [Numpy](https://numpy.org/) handles multi-dimensional arrays and provides an extensive library of mathematical functions. You may interact with this package directly or many higher-level pacakges you will interact with in the future will rely on `numpy` underneath. It is common practice to import `numpy` using an alias or shortened name:

In [ ]:
import numpy as np

In [ ]:
x = np.array([1, 2, 3, 4, 5])
x

Next we will import the `matplotlib.pyplot` package for simple plots.

In [ ]:
import matplotlib.pyplot as plt

## 1.3 loading data

In this section we will use numpy to read some scattering data in different formats.

In [ ]:
# This will download the file to our google colab workspace
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/100nmPinholeSphere.txt'
!wget 'content/' {url}

In [ ]:
# the argument to the loadtxt function, everything in (), is now a file path to the data
# this would either be a relative or absolute path on your computer if you were working with a local script
data = np.loadtxt('100nmPinholeSphere.txt', skiprows=12) # the skiprows argument allows us to skip the header in the file that is not part of the table format

In [ ]:
# the data will show up as an n x 4 array
# the second dimension refers to q, I(q), I(q) error, sigma q
data

We can perform simple transformations on the data, e.g. removing incoherent background, in a numpy array:

In [ ]:
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/n_coreshell.txt'
!wget 'content/' {url}
data = np.loadtxt('/content/n_coreshell.txt', skiprows=5)
plt.loglog(data[:,0], data[:,1], 'o')

## 1.4 functions and loops

## 1.5 EXERCISE: Practice basic Python concepts

Practice basic Python concepts learned in section 1 by removing the flat background from multiple SANS datasets. The first cell has been started for you and will load the files into your Google colab directory. The files are:  
x_coreshell_1.txt  
x_coreshell_2.txt  
x_coreshell_3.txt  
x_coreshell_4.txt  

Follow along with the commented prompts in the cell below to load the data into a dictionary, remove the background for each file, and plot the data on separate plots that shows the data before and after background removal.




In [ ]:
# run this cell to download all the data to your google colab file system
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/x_coreshell_1.txt'
!wget 'content/' {url}

url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/x_coreshell_2.txt'
!wget 'content/' {url}

url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/x_coreshell_3.txt'
!wget 'content/' {url}

url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/x_coreshell_4.txt'
!wget 'content/' {url}

In [ ]:
# start by creating a list of filenames, the first one is done for you

filenames = [
    '/content/x_coreshell_1.txt',

]

# let's create an empty dictionary where we will store the datasets we load
data_dict = {}

# now let's use numpy to load each of the datasets into an array and save them
# in the dictionary; we can use the filenames for the dictionary keys or you
# can use whatever other format you would like
# HINT: use a loop here so you don't have to type out loading each file!
# HINT: skip the first 9 rows of the file as these are part of the header


In [ ]:
# plot the datasets to see what you are working with
# again, try using a loop to cycle through all the datasets in the dictionary



In [ ]:
# create a function that removes the flat background from a dataset
# and then returns the subtracted dataset



In [ ]:
# now loop through the data again but use your function to remove the background
# and then store the subtracted data in a new dictionary



In [ ]:
# plot your subtracted datasets to see the results



Great work! Let's think about ways that the code above is specific to this dataset. In what ways would we need to change the code to make this more broadly applicable? How would you make your code more sophisticated in the future?

# 2.&nbsp;Simulating data with sasmodels

In [ ]:
# we need to install packages that are not standard on the colab environment
!pip install bumps==0.9.1 sasmodels==1.0.7 periodictable==1.6.1
!pip install sasdata==0.8.1
!pip install tol-colors
!pip install git+https://github.com/SasView/sasview.git@v5.0.6

# import sasmodels
import sasmodels
import sasmodels.core
import sasmodels.data
import sasmodels.bumps_model

# import bumps
import bumps
import bumps.fitters
import bumps.names
import bumps.fitproblem

# import sasdata
import sasdata
from sasdata.dataloader.loader import Loader

# import sasview for some legacy sesans functionality
import sas

import numpy as np
import os
import periodictable
import periodictable.nsf
import tol_colors as tc # colorblind safe color palettes


"""
import matplotlib.pyplot and set custom default settings for plots
"""
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.rc('font', size=14) # default fontsize
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=14)
plt.rc('legend', title_fontsize=14)
plt.rc('figure', titlesize=18)
plt.rc("figure", figsize=(5,5))
plt.rc("lines", linewidth=3)


# use colorblind safe colors:
plt.rc('axes', prop_cycle=plt.cycler('color', list(tc.tol_cset('muted'))))
try:
    plt.cm.register_cmap('tc_iridescent', tc.tol_cmap('iridescent'))
except:
    pass
plt.rc('image', cmap='tc_iridescent')

%config InlineBackend.figure_format = 'retina'

All available models can be accessed through the sasmodels `core` module. With `sasmodels.core.list_models()`:

## 2.1 sasmodels `kernel`
In order to access these models, we use sasmodels to build a `kernel`, which includes the theory for a specific model. The kernel includes all the information required to calculate the scattering intensity for a specific system, e.g., monodisperse spheres. We haven't yet told it anything about our data to fit or what format of data we want, i.e., 1D vs. 2D. This is just the theory part of the model.

Let's look at the [sphere](https://www.sasview.org/docs/user/models/sphere.html) form factor as our first example.

In [ ]:
# Create the sphere kernel


We can access the documentation and description found in the source files for each model:

In order to call our kernel, i.e., simulate scattering intensity, I(q), as a function of q, we need to create a callable version of the model. However, we need to give the kernel a dataset to know what type of data to return (1D vs. 2D). In this case, since we just want to simulate data, we can use an empty dataset using the `data` module in sasmodels:

In [ ]:
# Create an Empty 1D dataset of 1000 log-spaced q-values between 1e-4 inverse Angstroms and 1 inverse Angstroms.



To create the callable kernel, we will use the `direct_model` module:

Now we can call the kernal like a function in python and provide the required parameter values as arguments:

## 2.2 EXERCISE: Use a sasmodels `kernel` to explore a different model other than a sphere form factor.

What if you didn't have a system of spheres? What if you had cylinders, ellipsoids, or maybe you expect a fractal? So far we have only looked at sphere models, but SasView includes a wide range of shape and shape-independent models. You can find them listed in the SasView [documentation](https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/models/index.html) or pick a random one from the list generated by `sasmodels.core.list_models()`.

Pick a new model, create a callable kernel or model, and explore how the scattering signal is affected by changes to the parameter values.

## 2.3 polydispersity

Up until now, we have been working with monodisperse systems, however polydispersity can also be applied to certain parameters. Those parameters for 1D and 2D systems are identified in the kernel info:

In [ ]:
kernel = sasmodels.core.load_model("sphere")
kernel.info.parameters.pd_1d, kernel.info.parameters.pd_2d

The polydispersity is set using a special set of \*\_pd\_\* parameters:
* `*_pd*` is the width of the polydispersity and the exact definition will depend on the type of distribution specified; in this cause of a 'gaussian` distribution, it is defined as as $\frac{\sigma}{x_{mean}}$
* `*_pd_type*` defines the type of polydispersity distribution used
* `*_pd_n` corresponds to the the number of points in the distribution used by the calculation (Npts in the image below)
* `*_pd_nsigma` sets the range of the distribution sampled (Nsigmas in the image below)

For example, radius polydispersity could be defined by specifying **all four** of the pd parameters. With the default values, this would look like:
* `radius_pd = 0`
* `radius_pd_type = 'gaussian'`
* `radius_pd_n = 35`
* `radius_pd_nsigma = 3`

The available polydispersity distributions are defined in the SasView [documentation](https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/pd/polydispersity.html) but we can access them directly through the sasmodels `weights` module.

The 'gaussian' distribution is claculed by:

$$
f(x) = \frac{1}{Norm}\exp\left(-\frac{(x-x_{mean})^2}{2\sigma^2}\right)
$$

<img src=https://www.sasview.org/docs/_images/pd_gaussian.jpg>

*Image source: https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/pd/polydispersity.html*


Let's plot the default gaussian distribution but apply a pd value greater than 0 (apply polydispersity).

In [ ]:
distribution = "gaussian"

# The probability density function can be accessed using the weights.get_weights() method in sasmodels
pdf = sasmodels.weights.get_weights(
    distribution,
    n=35,               # _pd_n parameter
    width=0.2,          # this is equal to _pd parameter, or sigma/x_mean
    nsigmas=3,          # _pd_nsigma parameter
    value=100,          # corresponds to x_mean, or the value of the parameter to which polydispersity is applied, in this case it is the radius
    limits=[0, np.inf], # hard limit on the possible values (can't have negative radius)
    relative=True       # set to true if width is in proportion to the parameter value, or x_mean in this case
)

plt.errorbar(pdf[0], pdf[1], fmt='o-')
plt.xlabel("radius (Angstroms)")
plt.ylabel("probability density")
plt.title(f"polydispersity: {distribution} distribution")
plt.show()

We also notice that with the limits argument we can set hard limits on the possible values of the parameter across the distribution. Notice that when our radius is low and polydispersity is high, the distribution is cutoff at 0, the lower limit we have set.

In [ ]:
distribution = "gaussian"

# The probability density function can be accessed using the weights.get_weights() method in sasmodels
pdf = sasmodels.weights.get_weights(
    distribution,
    n=35,               # _pd_n parameter
    width=0.7,          # this is equal to _pd parameter, or sigma/x_mean
    nsigmas=3,          # _pd_nsigma parameter
    value=30,          # corresponds to x_mean, or the value of the parameter to which polydispersity is applied, in this case it is the radius
    limits=[0, np.inf], # hard limit on the possible values (can't have negative radius)
    relative=True       # set to true if width is in proportion to the parameter value, or x_mean in this case
)

plt.errorbar(pdf[0], pdf[1], fmt='o-')
plt.xlabel("radius (Angstroms)")
plt.ylabel("probability density")
plt.title(f"polydispersity: {distribution} distribution")
plt.show()

Gaussian is only one type of polydispersity distribution that we can apply. Others include 'boltzmann', 'uniform', 'lognormal', 'schulz' and more. The available distributions are defined in greater detail in the sasview [documentation](https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/pd/polydispersity.html) linked here.

Another example of the lognormal distribution is shown here:

In [ ]:
distribution = "lognormal"
pdf = sasmodels.weights.get_weights(
    distribution,
    n=35, # _pd_n parameter
    width=0.25, # this is equal to _pd parameter, or sigma/x_mean
    nsigmas=4, # _pd_nsigma parameter
    value=200, # corresponds to x_mean, or the value of the parameter to which polydispersity is applied, in this case it is the radius
    limits=[0, np.inf], # hard limit on the possible values (can't have negative radius)
    relative=True # set to true if width is in proportion to the parameter value, or x_mean in this case
)

plt.errorbar(pdf[0], pdf[1], fmt='o-')
plt.xlabel("radius (Angstroms)")
plt.ylabel("probability density")
plt.title(f"polydispersity: {distribution} distribution")
plt.show()

## 2.4 resolution

Up until now, we have not yet considered the effects of smearing on our data. We can look at the resolution specified on our callable kernel and it will be listed as a `Perfect1D` system unless we would like to apply a `Pinhole1D` or `Slit1D` resolution function to our data.



In [ ]:
q_min = 1e-4
q_max = 0.3
num_q = 1000
q = np.logspace(np.log10(q_min), np.log10(q_max), num_q)
data = sasmodels.data.empty_data1D(q)

# create callable kernel
model_name = "sphere"
kernel = sasmodels.core.load_model(model_name)
model = sasmodels.direct_model.DirectModel(data, kernel)

model.resolution

### pinhole smearing

<img width=600 src="https://drive.google.com/uc?id=1BDEiWSxrOwa17QBomEUSmphl5a2rcLu5">

*Image source: https://sestar.irb.hr/images/instrumenti/documents/52.pdf*


### slit smearing

<img width=600 src="https://saxs-igorcodedocs.readthedocs.io/en/stable/_images/SmearingGeometry.jpg">


*Source of slit smearing image: https://saxs-igorcodedocs.readthedocs.io/en/stable/Irena/Desmearing.html*


The resolution functions available are defined in the SasView [documentation](https://www.sasview.org/docs/user/qtgui/Perspectives/Fitting/resolution.html), but let's explore how the model is affected by pinhole and slit smearing.

In [ ]:
q_min = 0.00005
q_max = 0.5
num_q = 1000
q = np.logspace(np.log10(q_min), np.log10(q_max), num_q)
data = sasmodels.data.empty_data1D(q)

model_name = "sphere"
kernel = sasmodels.core.load_model(model_name)

The default resolution is set at Perfect1D. First calculate the scattering intensity with no smearing effects.

Now apply pinole resolution with a delta(q)/q value of 15%.

Finally, apply slit smearing with a slit length of 0.2 $Å^{-1}$.

Now create a plot to show the effects of pinhole and slit smearing. What do you notice about the effects of pinhole and slit smearing?

## 2.5 EXERCISE: Explore pinhole and slit smearing.

In the previous cell, we compared pinhole and slit smearing effects on the sphere model. But how is the data effected with varying slit length or delta(q)/q values?

Create two plots that demonstrates these effects. One plot will show the sphere form factor as a function of slit length (and slit width if you have time) and the other will show the sphere form factor as a function of delta(q)/q.

Remember to implement the python basics we've covered! Maybe loops or functions can help you explore many parameters?

## 2.6 combined models and structure factors

Models can be combined together by using '+' or '*' in the model name. For example, a binary system of spheres and cylinders could be created by loading a 'sphere+cylinder' model. What do you notice about the parameter names?

In [ ]:
q_min = 4e-4
q_max = 0.5
num_q = 1000
q = np.logspace(np.log10(q_min), np.log10(q_max), num_q)
data = sasmodels.data.empty_data1D(q)

kernel = sasmodels.core.load_model("sphere+cylinder")
kernel.info.parameters.defaults

Notice how now some parameters have an `A_` or `B_` prefix which correspond to the parameters for the `sphere` and `cylinder` models, respectively. The scale and background parameters are handled carefully. There are now `scale`, `A_scale`, and `B_scale` parameters and only a single `background` parameter. These are handled in the combined model as:

$I(q)=scale*(A\_scale *sphere + B\_scale*cylinder) + background$

Applying a structure factor is similar, but you use a `@` symbol instead. For example, a sphere with a hardsphere structure factor would be called with "sphere@hardsphere". Note how there are no prefixes on the parameters but rather we see the addition of the structure factor parameters. Note that a radius_effective_mode of 0 should be set in order to specify the effective radius. A mode of 1 will use the radius term for the sphere model to calculate it.

In [ ]:
# for radius effective modes, 0 is always unconstrained
# other modes of integers 1+ correspond to the list in each specific model
# for the sphere this is set to the radius (weighted average if polydisperse)
kernel = sasmodels.core.load_model("sphere")
kernel.info.radius_effective_modes

In [ ]:
# try it for the cylinder model
kernel = sasmodels.core.load_model("cylinder")
kernel.info.radius_effective_modes

## 2.7 EXERCISE: Use the sasmodels model to generate simulated data for a proposal.

In this exercise you will combine what you have learned above about sasmodels kernels, i.e., how they provide a callable function for calculating the scattering intensity from q-values and parameters using the defined model such as the spherical form factor, with the Python tools you learning in Morning Session 1, including how you can calculate sld and contrast from periodictable and plotting data.

You are submitting a proposal for beamtime at the **S**uper **A**wesome **N**ew **S**ANS (SANS) beamline where you would like to measure a size series and concentration series of polystyrene nanoparticles. For the proposal, you will provide simulated datasets of your expected results. Create two plots, one for the size series and one for a concentration series of a selected size, below using the sasmodels sphere model and hardsphere structure factor.

Some helpful information:
* Polystyrene
 * (C8H8)n, density = 1.05 g/mL
 * Particle diameters available: 20, 50, 100, 250, 500, 1000 nm (+/- 5%)
 * The particles have a polydispersity of 0.1 (gaussian)
* Solvent is heavy water
* expected q range is from 0.001 $nm^{-1}$ to 10 $nm^{-1}$
* estimate a dq/q at 15%



Create an empty dataset and callable kernel:

Apply the pinhole resolution:

Calculate the sld of the polystyrene nanoparticles and the solvent. TIP: earlier we installed the periodictable package. You can use the `periodictable.nsf.neutron_scattering` method (click [here](https://periodictable.readthedocs.io/en/latest/api/nsf.html#periodictable.nsf.neutron_scattering) for documentation) just like you would the [NIST sld calculator](https://www.ncnr.nist.gov/resources/activation/) (which is built using the periodictable package). Try calculating the SLD for polystyrene and D2O using this method!

In [ ]:
# look at the documentation to see what the returned parameters mean and which one is your sld
# remember to use indexing to select the value you need for your simulated datasets
sld_ps = periodictable.nsf.neutron_scattering(compound="C8H8", density=1.05, wavelength=6)
sld_ps

In [ ]:
sld_ps =
sld_d2o =


Create reference arrays of your particle radii and concentration (scale) to loop through during generation of your plots:

In [ ]:
radii =
concentrations =

Loop through the radii (don't forget about polydispersity!) and create the concentration series plot:

Loop through the concentrations (for a single particle size) and create the concentration series plot:

# 3.&nbsp;Fitting with the bumps package

The bumps fitting pacakge can be used to fit scattering data with the models we created in the previous section using sasmodels. In this section we will learn how to create the sasmodels `bumps_model.Model` and `bumps_model.Experiment` which interface with bumps, apply parameter constraints, and fit a single scattering curve. Finally, we will cover how to perform simulataneous fitting, which is helpful if you have two datasets, e.g. USANS and SANS, for the same sample and need to hold parameters (e.g. sphere radius) constant across the two datasets and their fits.

## 3.1 sasmodels `Model` and parameter constraints
We have learned how to generate a kernel using sasmodels, which calculated the theoretical scattering intensity as a function of the q-values and parameter values. In order to use our kernel for data fitting, we must create a model that interfaces with the bumps fitting library. This is done by creating a `Model` and `Experiment` from the sasmodels `bumps_model` module.

The `Model` is similar to a callable kernel or direct model we used before, however it will now explicitly list the polydispersity parameters and it creates a bumps `Parameter` object for each parameter of the model.

Each of the parameters is an attribute of the `Model` and are also a bumps `Parameter` type.

A parameter object has atributes including the limits, fitting range (when specified as a fittable parameter), the value and even bounds that can follow a distribution.

What are the `limits` of the sphere radius?

What about its value?

We can directly change this value of the parameter...

...or we can create a dictionary of parameters to pass to the `Model` function.

The parameter is assumed fixed unless we tell it otherwise, i.e., provide a fitting range or bounds.

Set a fitting range from 0 to 300 Angstroms for the radius:

This information is now stored in the `bounds` of the parameter:

When using the range method, there is a uniform probability of seeing any value within the bounds specified. However, what if you had some confidence of your particle radius from another characterization method? You can use other distributions, such as a Normal distribution, to still allow the parameter to vary slightly but penalize values that stray too far from the set value.

In [ ]:
model.radius.bounds = bumps.bounds.Normal(mean=50, std=1)
radius_values = np.arange(45, 55, 0.1)
pdf = model.radius.bounds.dist.pdf(radius_values)
plt.plot(radius_values, pdf)
plt.ylim(0, 0.5)
plt.ylabel("Probability Density")
plt.xlabel("Radius (Angstroms)")

The current value of all parameters can always be called using `model.state()`:

In [ ]:
model.state()

As mentioned before, we need not only a `bumps_model.Model` object but also a `bumps_model.Experiment` to interface with the bumps fitting module. Since the `Experiment` will combine the model information, including the kernel and parameter values, together with the scattering data, let's first look at how to import scattering data.

## 3.2 Importing scattering data

Data can be loaded using the SasView data methods which can read common data formats of 1D or 2D SANS data and 1D SESANS data from many facilities. These are described in the documentation [here](https://www.sasview.org/docs/user/qtgui/MainWindow/data_formats_help.html) in greater detail. Today we will work with .txt files that have three columns: q, I(q), dI(q).

In [ ]:
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/1umSlitSmearSphere.ABS'
!wget 'content/' {url}

# the Loader can load a single or multiple files, so it creates a list of the data objects
# we will just select the first object in the list and work with single files for now
loader = Loader()
loaded_data_list = loader.load('/content/1umSlitSmearSphere.ABS')
data = loaded_data_list[0]

In [ ]:
type(data)

The units are assumed to be $Å^{-1}$ for q and $cm^{-1}$ for I(q), and are noted in sasmodels `Data1D` object attributes:

In [ ]:
data.x_unit, data.y_unit

You can also directly create a `sasmodels.data.Data1D` instance if your data is stored in another format, such as a numpy array.

In [ ]:
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/1d_data/100nmPinholeSphere.txt'
!wget 'content/' {url}
data_np = np.loadtxt("100nmPinholeSphere.txt", skiprows=12)
data_np

In [ ]:
data = sasmodels.data.Data1D(x=data_np[:,0], y=data_np[:,1], dy=data_np[:,2], dx=data_np[:,3])
sasmodels.data.plot_data(data)

SasView also supports other types of data, such as SESANS:

In [ ]:
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/sesans_data/sphere2micron.ses'
!wget 'content/' {url}

# the Loader can load a single or multiple files, so it creates a list of the data objects
# we will just select the first object in the list and work with single files for now
loaded_data_list = Loader().load(url.split('/')[-1])
data = loaded_data_list[0]

## 3.3 sasmodels `Experiment`

As previously mentioned, the `Experiment` will combine the model information, including the kernel and parameter values, together with the scattering data. It is similar to a callable verison of the kernel, but will interface with the bumps fitting pacakge to enable fitting of the scattering data using the kernel and parameter information.

Let's load the '100nmPinoholeSphere.txt` dataset again. If you are using a numpy area, remember to create the sasmodels.data.Data1D object!

In [ ]:
data = np.loadtxt("/content/100nmPinholeSphere.txt", skiprows=12)
data = sasmodels.data.Data1D(x=data[:,0], y=data[:,1], dy=data[:,2], dx=data[:,3])

Create the `kernel`, `Model`, and then the `Experiment`:

The experiment allows us to access the typical plots we see in SasView, including the model data plotted against the theoretical data and the residuals. We can see right away that the default parameters are far from the system captured in the experimental data.

In [ ]:
plt.figure(figsize=(10,5))
experiment.plot()


## 3.4 EXERCISE: Finding good starting parameters for a fit.

Before we begin fitting, try to find better starting parameters in the model, similar to what we would do in SasView before we start the fit.

## 3.5 Single Curve Fitting

Now that we have found a good starting place for the fit, create parameter constraints and set which parameters will be allowed to vary during the fit by setting a range.

In [ ]:
params = {
    "radius": 500,
    "background": 0,
    "scale": 0.03
}
model = sasmodels.bumps_model.Model(model=kernel, **params)

model.scale.range(0, 1)
# model.radius.range(0, 1000)

experiment = sasmodels.bumps_model.Experiment(data=data, model=model)

To start the fit, we need to create a bumps `FitProblem` which translates our sasmodels information in `Experiment` into bumps where we can access different optimizers and view our fitting results.

To run the fit, we call `bumps.fitters.fit`:

We can see the summary of the results and current state or value of all the parameters in the `FitProblem`:

In [ ]:
print(problem.summarize())

In [ ]:
problem.fitness.model.state()

In [ ]:
problem.fitness.theory()

We can also see the final fit values and uncertainties in `results`:

In [ ]:
results.x

In [ ]:
results.dx

In [ ]:
problem.labels()

We also see that these values are updated in the original model:

In [ ]:
model.radius.value

And the information is carried through the experiment:

In [ ]:
plt.figure(figsize=(10,5))
experiment.plot()

The default optimizer for the bumps fit is the Levenberg-Marquardt method, but there are many available as described in greater detail in the Bumps [documentation](https://bumps.readthedocs.io/en/latest/guide/optimizer.html) with additional fitting options.

For Levenberg-Marquardt, the steps, f(x) tolerance and x tolerance from the GUI options can be set with the `steps`, `ftol` and `xtol` arguments in the `fit` method.

<img src="https://bumps.readthedocs.io/en/latest/_images/fit-lm.png">

*Image source: https://bumps.readthedocs.io/en/latest/guide/optimizer.html*

In [ ]:
results = bumps.fitters.fit(problem, method='lm', steps=1000, ftol=1.5e-08, xtol=1.5e-08, verbose=True)

plt.figure(figsize=(15,5))
problem.plot()

The DREAM is a Markov chain monte carlo fitting algorithm that is less likely to get trapped in local minima as LM occasionally does, but it is more computationally expensive than LM.

The samples, burn-in steps, population, initializer, thinning, and steps can be set with `burn`, `pop`, `init`, `thin` and `steps` arguments.

<img src="https://bumps.readthedocs.io/en/latest/_images/fit-dream.png">

*Image source: https://bumps.readthedocs.io/en/latest/guide/optimizer.html*

In [ ]:
results = bumps.fitters.fit(problem, method='dream', steps=1000, verbose=True)

plt.figure(figsize=(15,5))
problem.plot()

DREAM also provides additional uncertainty analysis:

## 3.6 Simultaneous fitting

It is also possible to fit two or more datasets simulataneous, i.e., keep parameters equivalent across multiple datasets as they vary during the fit. This can be useful if, for example, you collected both USANS and SANS data for the same sample and would like to fit a sphere model across both datasets. Instead of desmearing the data and merging the datasets prior to fitting them, each dataset can be fit with it's own smeared model.

Two datasets will be created in the next cell for this exercise. We will be looking at polystyrene nanoparticles in heavy water.

In [ ]:
# @title
# generate polystyrene np sans and usans data

q_min = 0.002
q_max = 0.3
num_q = 100
q = np.logspace(np.log10(q_min), np.log10(q_max), num_q)
data = sasmodels.data.empty_data1D(q, resolution=0.05)
kernel = sasmodels.core.load_model("sphere")
call_kernel = sasmodels.direct_model.DirectModel(data, kernel)

call_kernel.simulate_data(background=0.096, noise=15, radius=5295, radius_pd=0.05, radius_pd_n=35, radius_pd_nsigmas=35, radius_pd_type='gaussian', scale=0.05, sld=periodictable.nsf.neutron_sld(compound="C8H8", density=1.05)[0], sld_solvent=periodictable.nsf.neutron_sld(compound="D2O", density=1.11)[0])

# plt.errorbar(data.x, call_kernel.Iq, yerr=call_kernel.dIq, fmt='o')
# plt.xscale('log')
# plt.yscale('log')

data_save = np.vstack((data.x, call_kernel.Iq, call_kernel.dIq, data.dx)).T
np.savetxt("PolystyreneNP2_SANS.txt", data_save, header="q (1/Ang), I(q) (1/cm), dI(q), dq")

q_min = 0.00008
q_max = 0.003
num_q = 30
q = np.logspace(np.log10(q_min), np.log10(q_max), num_q)
data = sasmodels.data.empty_data1D(q)
kernel = sasmodels.core.load_model("sphere")
call_kernel = sasmodels.direct_model.DirectModel(data, kernel)
call_kernel.resolution = sasmodels.resolution.Slit1D(data.x, 0.117)

call_kernel.simulate_data(background=0, noise=20, radius=5295, radius_pd=0.05, radius_pd_n=35, radius_pd_nsigmas=35, radius_pd_type='gaussian', scale=0.05, sld=periodictable.nsf.neutron_sld(compound="C8H8", density=1.05)[0], sld_solvent=periodictable.nsf.neutron_sld(compound="D2O", density=1.11)[0])

# plt.errorbar(data.x, call_kernel.Iq, yerr=call_kernel.dIq, fmt='o')
# plt.xscale('log')
# plt.yscale('log')

data_save = np.vstack((data.x, call_kernel.Iq, call_kernel.dIq)).T
np.savetxt("PolystyreneNP2_USANS.txt", data_save, header="q (1/Ang), I(q) (1/cm), dI(q), dq")

In [ ]:
sans_data = np.loadtxt("PolystyreneNP2_SANS.txt")
sans_data = sasmodels.data.Data1D(x=sans_data[:,0], y=sans_data[:,1], dy=sans_data[:,2], dx=sans_data[:,3])
usans_data = np.loadtxt("PolystyreneNP2_USANS.txt")
usans_data = sasmodels.data.Data1D(x=usans_data[:,0], y=usans_data[:,1], dy=usans_data[:,2])


Next, setup the kernel:

Determine the sld of the polystyrene nanoparticles and the heavy water solvent.

In [ ]:
sld_ps = periodictable.nsf.neutron_sld(compound="C8H8", density=1.05)[0]
sld_d2o = periodictable.nsf.neutron_sld(compound="D2O", density=1.11)[0]

Create a single model and specify which parameters will be fit:

Create two separate experiments for the sans and usans datasets. The resolution dq was extracted from the SANS dataset, but the USANS dataset is slit smeared with a length of 0.117 inverse Angstroms. The resolution can be applied to the experiment, similar to the callable kernel.

We can create a `MultiFitProblem` in bumps by passing a list of experiments rather than a single one. Then fit as normal and plot the results.

In [ ]:
plt.rc("figure", figsize=(10,5)) # temporarily change default figure size
problem.plot()


Let's make a single plot with all the datasets:

In [ ]:
plt.rc("figure", figsize=(15,5))

sans_experiment.plot()
usans_experiment.plot()



We can also confirm the fit parameters are equivalent across both datasets:

In [ ]:
sans_experiment.parameters()["radius"].value, usans_experiment.parameters()["radius"].value, model.radius.value

## 3.7 Saving your fit

Remember to export your results when you are done fitting! You can do this in any format you like. I will typically export my fitting results and uncertainties along side the entire set of parameters for my model. When I am plotting my fit alongside my data, I can easily load the entire dictionary of parameters, simulate the model, and then plot alongside my experimental data.

Here I will show an example of how to export the dictionary of model parameters in model.state() to a csv file.

In [ ]:
import csv

with open('fit_parameters_sphere_simultaneous.csv','w') as f:
    w = csv.writer(f)
    w.writerows(model.state().items())
f.close()

# 4.&nbsp;Final Exercise, Discussion, Q&A

What questions do you still have? What about your own dataset? Use this time to try loading in some scattering data of your own and implementing a single fit. I'll be walking around to answer questions and help everyone get started implementing their own custom fitting protocols.

You can also follow along with the dataset below if you prefer. It is another example of fitting but for SESANS data! The protocol is the same as fitting regular SAS data. However, remember it's the data object that tells sasmodels what you need and the appropriate resolution. In this case, a SESANS data object (a type of Data1D) will tell sasmodels to compute the Hankel transform and provide you the model data in correlation space.

Run this cell to load the SESANS dataset from the sasview example data.

In [ ]:
url = 'https://raw.githubusercontent.com/SasView/sasview/main/src/sas/example_data/sesans_data/sphere2micron.ses'
!wget 'content/' {url}

# the Loader can load a single or multiple files, so it creates a list of the data objects
# we will just select the first object in the list and work with single files for now
loaded_data_list = Loader().load(url.split('/')[-1])
data = loaded_data_list[0]

Now setup a single curve fit just like we did before with the SAS data. Set up your kernel, parameters, bumps model, experiment, and fit problem. Don't forget to plot your results! You may have to play around with the parameters to get a good starting place. How does the plot and data compare to the SAS results we saw before?

#5.&nbsp;Bonus Section: 2D datasets

We can apply a similar approach to what we learned above for 1D datasets to 2D scattering datasets.

In order to generate an empty 2D dataset, we need to define the maximum q at the edges and the number of q-values (could think of this as number of pixels along the x and y direction of a square detector):

In [ ]:
q_max = 0.4 # limit along edge of detector
num_q = 100 # number of q values along x and y directions, use even number
q = np.linspace(q_max*0.5/(int(num_q/2)-0.5), q_max, int(num_q/2))
q = np.hstack((-1*np.flip(q), q))
data2d = sasmodels.data.empty_data2D(qx=q, qy=q, resolution=0)

At each "pixel" location, the true q-value has qx and qy components, and so the 2D empty dataset is structured with a `qx_data` array and `qy_data` array that is used to calculate the `q_data` array as $q=(qx^2+qy^2)^{\frac{1}{2}}$.



In [ ]:
data2d.qx_data[:num_q]

In [ ]:
data2d.qx_data[num_q:num_q*2]

In [ ]:
data2d.qy_data[:num_q]

In [ ]:
data2d.qy_data[num_q:num_q*2]

Let's plot the 2D scattering intensity for oriented [cylinders](https://www.sasview.org/docs/user/models/cylinder.html).

In [ ]:
kernel = sasmodels.core.load_model("cylinder")
call_kernel = sasmodels.direct_model.DirectModel(data2d, kernel)
kernel.info.parameters.defaults

The `phi` and `theta` parameters now become relevant when we are in the 2D space. The SasView cylinder [documentation](https://www.sasview.org/docs/user/models/cylinder.html) defines these angles as shown below relative to the detector plane.

<img src="https://www.sasview.org/docs/_images/cylinder_angle_definition.png">

*Image source: https://www.sasview.org/docs/user/models/cylinder.html*


Plot the model with the default parameter values:

In [ ]:
Iq = call_kernel()

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap='jet')
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()


Plot the 2D scattering data for the two projections shown below:

<img src="https://www.sasview.org/docs/_images/cylinder_angle_projection.png">

*Image source: https://www.sasview.org/docs/user/models/cylinder.html*

In [ ]:
Iq = call_kernel(phi=0, theta=90)

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap=tc.tol_cmap('sunset'))
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()

In [ ]:
Iq = call_kernel(phi=90, theta=45)

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap=tc.tol_cmap('sunset'))
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()

What happens when both `phi` and `theta` are set to 0 degrees? What do the neutrons "see"?

In [ ]:
Iq = call_kernel(phi=0, theta=0)

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap=tc.tol_cmap('sunset'))
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()

We can also apply pinhole resolution (no slit smearing yet) but with a different approach. Here we apply the dq/q value directly in the resolution argument of the `empty_data2d` method:

In [ ]:
# apply pinhole resolution when generating empty dataset
data2d = sasmodels.data.empty_data2D(qx=q, qy=q, resolution=0.05)

call_kernel = sasmodels.direct_model.DirectModel(data2d, kernel)

Iq = call_kernel()

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap=tc.tol_cmap('sunset'))
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()

In [ ]:
# apply pinhole resolution when generating empty dataset
data2d = sasmodels.data.empty_data2D(qx=q, qy=q, resolution=0.15)

call_kernel = sasmodels.direct_model.DirectModel(data2d, kernel)

Iq = call_kernel()

fig, ax = plt.subplots()
im = ax.pcolormesh(data2d.qx_data.reshape(2*int(num_q/2), -1), data2d.qy_data.reshape(2*int(num_q/2), -1), np.log10(Iq.reshape(2*int(num_q/2), -1)), cmap=tc.tol_cmap('sunset'))
fig.colorbar(im, label='log$_{10}$[I(q)]')

ratio = 1.0
x_min, x_max = (np.min(data2d.qx_data), np.max(data2d.qx_data))
y_min, y_max = (np.min(data2d.qy_data), np.max(data2d.qy_data))
ax.set_aspect(abs((x_max-x_min)/(y_min-y_max))*ratio)

plt.xlabel("q$_x$ ($\AA^{-1}$)")
plt.ylabel("q$_y$ ($\AA^{-1}$)")

plt.show()